In [2]:
import pandas as pd
from tqdm.notebook import tqdm
import snscrape.modules.twitter as sntwitter

# Twitter Scraping using snscraper

In [3]:
tweets=[]
scraper = sntwitter.TwitterSearchScraper("#WPL2023")
i=0
for tweet in tqdm(scraper.get_items(),total=2000):
    i=i+1
    data=[tweet.content]
    tweets.append(data)
    if(i==2000):
        break

df=pd.DataFrame(tweets,columns=['text'])
df.to_csv("#WPL2023.csv")

  0%|          | 0/2000 [00:00<?, ?it/s]

C:\Users\anik3\AppData\Local\Temp\ipykernel_1948\520126653.py:6: FutureWarning: content is deprecated, use rawContent instead
  data=[tweet.content]


# Cleaning Tweets before generating summary for dataset

### PreProcessing Functions

In [115]:
from langdetect import detect
def detectlang(t):
    lang=''
    try:
        lang=detect(t)
    except:
        print("Exception")
    return lang

import re
def remove_url(text):
    return re.sub('http://\S+|https://\S+', '', text)
    # return pattern.sub(r'',t)

import emoji

punctuation = "!”$%’()*+-/:;<=>[\]^_`{|}~•@#"
def removepunc(text):
    return text.translate(str.maketrans('','',punctuation))

def removeEscapeSequences(x):
    return x.replace("\n","")

def removeAmpercent(x):
    return x.replace("&", "and")


def removeHashTags(text):
    return re.sub('#[A-Za-z0-9]+', '', text)


### AyodhyaVerdict

In [20]:
df=pd.read_csv("#AYODHYAVERDICT_tweets.csv")
df['text']

0      Ahead of #AyodhyaVerdict, appeal to everyone t...
1      We respect the Hon’ble Supreme Court’s #Ayodhy...
2      This is the last prayer offered at #BabriMasji...
3      “गिरा अरथ जल बीचि सम कहिअत भिन्न न भिन्न। \nबं...
4      "There is a profound reason Maharshi Valmiki t...
                             ...                        
195    As a step to impress Sonia Gandhi&amp; Congres...
196    By #AYODHYAVERDICT,ALL pending issues like-#Tr...
197    கோவில் கொள்ளைக்காரர்களின் கூடாரம்! ~ கலைஞர்\n\...
198    Just bcoz BJP got 303 seats in LS everything l...
199    That's Barrister @asadowaisi . Your name will ...
Name: text, Length: 200, dtype: object

In [28]:
# t="ajkdjf d https://hdfjhd.com/djkjdk njfghdjfg"
# t=remove_url(t)
# t

In [29]:
for i in range(200):
    df['text'][i]=remove_url(df['text'][i])
    df['text'][i]=emoji.demojize(df['text'][i])
    df['text'][i]=removepunc(df['text'][i])
    df['text'][i]=removeEscapeSequences(df['text'][i])
    df['text'][i]=removeAmpercent(df['text'][i])
    df['text'][i]=removeHashTags(df['text'][i])


    # df['text'][i]=removeapostophes(df['text'][i])




C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\2432296291.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=remove_url(df['text'][i])
C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\2432296291.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=emoji.demojize(df['text'][i])
C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\2432296291.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=remov

In [30]:
c=0
newdf=[]
temp=""
for i in range(200):
    if (detectlang(df['text'][i]))=='en':
        temp=temp+df['text'][i]
        c=c+1
    if c==10:
        c=0
        newdf.append(temp)
        temp=""

newdf.append(temp)

### Pegasus Model for Summary Generation

In [32]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

In [33]:
# Load tokenizer 
tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")
# Load model 
model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")

In [36]:
def generate_summary(text):
    tokens = tokenizer(text, truncation=True, padding="longest", return_tensors="pt")
    summary = model.generate(**tokens,min_new_tokens=256,no_repeat_ngram_size =1)

    # Decode summary
    t=""
    t=tokenizer.decode(summary[0])
    return t

In [38]:
dict={}
input=[]
output=[]

In [94]:
for i in range (len(newdf)):
    t=""
    # print(newdf[i])
    input.append(newdf[i])
    # print("summary")
    t=generate_summary(newdf[i])
    # print(t)
    output.append(t)
    # print()
    # print()
    # print()
    

In [56]:
def preprocess(df):
    for i in range(df.shape[0]):
        df['text'][i]=remove_url(df['text'][i])
        df['text'][i]=emoji.demojize(df['text'][i])
        df['text'][i]=removepunc(df['text'][i])
        df['text'][i]=removeEscapeSequences(df['text'][i])
        df['text'][i]=removeAmpercent(df['text'][i])
        df['text'][i]=removeHashTags(df['text'][i])

In [58]:
def buildsetoften(df):
    c=0
    newdf=[]
    temp=""
    for i in range(df.shape[0]):
        if (detectlang(df['text'][i]))=='en':
            temp=temp+df['text'][i]
            c=c+1
        if c==10:
            c=0
            newdf.append(temp)
            temp=""

    newdf.append(temp)
    return newdf

### CancellAllBlueTicks

In [46]:
df=pd.read_csv("#cancelallBlueTicksinIndia_tweets.csv")
df['text']

0      ये नया तथ्य सामने आया है, पैसे लेकर ब्लू टिक द...
1      भारत का जातिवादी मीडिया 'ट्विटर बनाम दलित' बता...
2      ट्विटर पर क्‍यों छिड़ा है ट्विटर और दलित कार्य...
3      @dilipmandal भारत का जातिवादी मीडिया 'ट्विटर ब...
4      #cancelallBlueTicksinIndia has been the top tr...
                             ...                        
193                 @sanjuydv #cancelallBlueTicksinIndia
194    प्रिय ट्विटर.\nहम किसी ब्लू टिक के मोहताज नही ...
195    जो @Twitter @verified ब्लू टिक होल्डर हमारे सा...
196    ट्विटर पर क्‍यों छिड़ा है ट्विटर और दलित कार्य...
197    One man One vote,\nOne vote One value...\n\nसम...
Name: text, Length: 198, dtype: object

In [57]:
preprocess(df)

C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=remove_url(df['text'][i])
C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=emoji.demojize(df['text'][i])
C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=remov

In [95]:
newdf=buildsetoften(df)

In [63]:
for i in range (len(newdf)):
    t=""
    # print(newdf[i])
    input.append(newdf[i])
    # print("summary")
    t=generate_summary(newdf[i])
    # print(t)
    output.append(t)
    # print()
    # print()
    # print()

c:\Users\anik3\anaconda3\lib\site-packages\transformers\generation\utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 64 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


### JNUProtest

In [75]:
df=pd.read_csv("#jnuprotest_tweets.csv")
print(df.text)
preprocess(df)
newdf=buildsetoften(df)
for i in range (len(newdf)):
    t=""
    # print(newdf[i])
    input.append(newdf[i])
    # print("summary")
    t=generate_summary(newdf[i])
    # print(t)
    output.append(t)

0      Statue of Unity - 3000 cr\n\nCost of printing ...
1      "JNU gets more funds than any other universiti...
2      Statue of Unity - 3000 cr\n\nCost of printing ...
3      Came to Delhi in 2004..worked in a call centre...
4      "JNU gets more funds than any other universiti...
                             ...                        
195    JNU fee hike would forever shut the doors of t...
196    I am a student.\nI want my right.\nI want educ...
197    To destroy a country, U don't need to use nuke...
198    To destroy a country, U don't need to use nuke...
199    While Students of #JNU have been protesting fo...
Name: text, Length: 200, dtype: object


C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=remove_url(df['text'][i])
C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=emoji.demojize(df['text'][i])
C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=remov

### Kashmir Tweets

In [76]:
df=pd.read_csv("#Kashmir_tweets.csv")
print(df.text)
preprocess(df)
newdf=buildsetoften(df)
for i in range (len(newdf)):
    t=""
    # print(newdf[i])
    input.append(newdf[i])
    # print("summary")
    t=generate_summary(newdf[i])
    # print(t)
    output.append(t)

0      First forceful occupation of #Kashmir and now ...
1      Snowfall in #Kashmir #AJK 🍁https://t.co/KdL1lR...
2      مرے مالک مرے رہبر مدد کر \nکہ ہے کشمیر خوں سے ...
3      06 نومبر یوم شہدائے جموں\n\n#Kashmir #KashmirS...
4      Snowfall in #Kashmir #AJK 🍁https://t.co/KdL1lR...
                             ...                        
195    Pakistan is deliberately encouraging anti-Hind...
196    #Kashmir: Is national security not a good grou...
197    A house destroyed By Indian Shelling in AJK, l...
198    This is it. Simple and straight!\nAll that tal...
199    مقبوضہ کشمیر میں کرفیو کا 94واں روز، عالمی برا...
Name: text, Length: 200, dtype: object


C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=remove_url(df['text'][i])
C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=emoji.demojize(df['text'][i])
C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=remov

### KejriwalMustResign

In [77]:
df=pd.read_csv("#KejriwalMustResign_tweets.csv")
print(df.text)
preprocess(df)
newdf=buildsetoften(df)
for i in range (len(newdf)):
    t=""
    # print(newdf[i])
    input.append(newdf[i])
    # print("summary")
    t=generate_summary(newdf[i])
    # print(t)
    output.append(t)

0      #AYODHYAVERDICT\n#JaiBhimJaiPeriyar\n#WhySoBor...
1      पंजाब में "आप" द्वारा पराली जाने पर और दिल्ली ...
2      Why no-body is talking or writing ✍️ about sol...
3            #KejriwalMustResign, sound like blame game.
4      #KejriwalMustResign, if that solves all the pr...
                             ...                        
195    #KejriwalMustResign \n\nThis is the account of...
196    IRONY OF THE NATION's CAPITAL\n\n#ThursdayThou...
197    सर्वोच्च न्यायालय ने भी मेरे  द्वारा उठाये गए ...
198    पंजाब में "आप" द्वारा पराली जाने पर और दिल्ली ...
199    पंजाब में "आप" द्वारा पाराली जलाने और दिल्ली म...
Name: text, Length: 200, dtype: object


C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=remove_url(df['text'][i])
C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=emoji.demojize(df['text'][i])
C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=remov

### INDvAUS

In [82]:
df=pd.read_csv("#INDvAUS.csv")
print(df.text)
preprocess(df)
newdf=buildsetoften(df)
for i in range (len(newdf)):
    t=""
    # print(newdf[i])
    input.append(newdf[i])
    # print("summary")
    t=generate_summary(newdf[i])
    # print(t)
    output.append(t)

0       #INDvAUS \n\n#jadeja #IndvsAus #INDvAUS\n#INDv...
1       #INDvAUS\nThe way @RishabhPant17 plays test cr...
2                                     #INDvAUS\n👏🏻👏🏻👏🏻💙💙💙
3       All the more reason to create a pitch like thi...
4       #BGT2023 #BorderGavaskarTrophy2023 \n#INDvAUS ...
                              ...                        
1495    Kya lagta hai aapko 'Who will score more fours...
1496    Sometims you get those. Smith cops an absolute...
1497    IND vs AUS 3rd Test Live: Ravindra Jadeja gets...
1498    If there are no communication devices allowed ...
1499    4th wicket for Sir Jadeja but Australia have g...
Name: text, Length: 1500, dtype: object


C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=remove_url(df['text'][i])
C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=emoji.demojize(df['text'][i])
C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=remov

### chatgpt

In [86]:
df=pd.read_csv("#chatgpt.csv")
print(df.text)
preprocess(df)
newdf=buildsetoften(df)
for i in tqdm(range (len(newdf)),total=len(newdf)):
    t=""
    # print(newdf[i])
    input.append(newdf[i])
    # print("summary")
    t=generate_summary(newdf[i])
    # print(t)
    output.append(t)

0      @LearningengAki #ChatGPT gpt-3.5-turbo を試さずにねる...
1      🌠#Interstellar pictures again today \n\n💥Impre...
2      #ChatGPT recently passed a Wharton School MBA ...
3      Google is in trouble\n\nAsked Google to answer...
4      ChatGPT API is now publicly available. Also, w...
                             ...                        
495    $ITM Awaiting Buy Signal based off 10 trades, ...
496    $EFX Awaiting Short Signal based off 11 signal...
497    $HYLB Awaiting Short Signal based off 6 trades...
498    $URI Awaiting Buy Signal based off 11 signals ...
499    $IAI Awaiting Short Signal based off 7 trades,...
Name: text, Length: 500, dtype: object


C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=remove_url(df['text'][i])
C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=emoji.demojize(df['text'][i])
C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=remov

  0%|          | 0/47 [00:00<?, ?it/s]

c:\Users\anik3\anaconda3\lib\site-packages\transformers\generation\utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 64 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


### SARSCov2

In [87]:
df=pd.read_csv("#SARSCoV2.csv")
print(df.text)
preprocess(df)
newdf=buildsetoften(df)
for i in tqdm(range (len(newdf)),total=len(newdf)):
    t=""
    # print(newdf[i])
    input.append(newdf[i])
    # print("summary")
    t=generate_summary(newdf[i])
    # print(t)
    output.append(t)

0      There were reports that TV Journalist Savannah...
1      If COVID, SARS-CoV-2 is “over,” why is it stil...
2      @JannikaJahn @HongMathias don't you feel the d...
3      Mask have to work. Otherwise #SARSCOV2 is spre...
4      @brownecfm This is what concerns me the most: ...
                             ...                        
495    Met een enorm doden aantal.\n\n#COVID19 \n#SAR...
496    Is there anyone in the U.K. with an objective ...
497    Und dann kommen wir wahrscheinlich zur größten...
498    On claims that DOE concluded #SARSCoV2 was a m...
499    Origine du #SARSCov2 = la thèse de l’accident ...
Name: text, Length: 500, dtype: object


C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=remove_url(df['text'][i])
C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=emoji.demojize(df['text'][i])
C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=remov

  0%|          | 0/32 [00:00<?, ?it/s]

c:\Users\anik3\anaconda3\lib\site-packages\transformers\generation\utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 64 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


### iphone 14

In [88]:
df=pd.read_csv("#iphone14.csv")
print(df.text)
preprocess(df)
newdf=buildsetoften(df)
for i in tqdm(range (len(newdf)),total=len(newdf)):
    t=""
    # print(newdf[i])
    input.append(newdf[i])
    # print("summary")
    t=generate_summary(newdf[i])
    # print(t)
    output.append(t)

0      UQモバイル UQ応援割 学割 2022年-2023年の契約がお得 U18は のりかえ MN...
1      Apple iPhone 14 128GB Estelar 5G saindo por\n🔥...
2      iPhone14 のりかえ MNP 1円ではないが、総額3万円程度で購入できるように \n\...
3      #breaking #Apple #NewProducts #iPhone14 #color...
4      Apple iPhone 14 (128 GB) Estelar em oferta por...
                             ...                        
495    【Bic Style ららぽーと愛知東郷店】\n\n✨ #iPhone14 #ProMax ...
496    【本日のPR】\n\nアイプラス京都アバンティ店では、劣化したバッテリーの交換修理や割れてし...
497    ☆買取情報☆\n\n4/9まで、\nスマホ、タブレットが買取価格10%UP中！\n\n未使用...
498    2/28\nTKモバイル \n単価変動アリ\n\nRT❤️提出\n500UP\n#13pro...
499    【のりかえ】 MNP iPhone14 iPhone13 一括 価格 キャッシュバック 機種...
Name: text, Length: 500, dtype: object


C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=remove_url(df['text'][i])
C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=emoji.demojize(df['text'][i])
C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=remov

  0%|          | 0/24 [00:00<?, ?it/s]

c:\Users\anik3\anaconda3\lib\site-packages\transformers\generation\utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 64 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


### openai

In [89]:
df=pd.read_csv("#openai.csv")
print(df.text)
preprocess(df)
newdf=buildsetoften(df)
for i in tqdm(range (len(newdf)),total=len(newdf)):
    t=""
    # print(newdf[i])
    input.append(newdf[i])
    # print("summary")
    t=generate_summary(newdf[i])
    # print(t)
    output.append(t)

0       Exploring OpenAI’s Groundbreaking AI Models an...
1       Switched a sample app from #OpenAI web APIs to...
2       Investors are going nuts for #ChatGPT-ish arti...
3       Après une dizaine de jours d'échanges avec #Ch...
4       Afternoon Concept:\n#artistsontwitter #ArtistO...
                              ...                        
1495    #JPMorgan è solo l’ultima società a limitare l...
1496    CHATGPT BECOMES AN INTEGRAL PART OF THE INTERN...
1497    Brad and Mitchell's forbidden love had grown s...
1498    Ethereum developers have taken another step to...
1499    🛸 #Metaverse \n\nSnapchat has released 'My AI'...
Name: text, Length: 1500, dtype: object


C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=remove_url(df['text'][i])
C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=emoji.demojize(df['text'][i])
C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=remov

  0%|          | 0/120 [00:00<?, ?it/s]

c:\Users\anik3\anaconda3\lib\site-packages\transformers\generation\utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 64 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


### Ipl2022

In [90]:
df=pd.read_csv("#ipl2022.csv")
print(df.text)
preprocess(df)
newdf=buildsetoften(df)
for i in tqdm(range (len(newdf)),total=len(newdf)):
    t=""
    # print(newdf[i])
    input.append(newdf[i])
    # print("summary")
    t=generate_summary(newdf[i])
    # print(t)
    output.append(t)

0       @GappaCricket Did any one expected #GT to win ...
1       Bowling ka style \nGully cricket practice matc...
2       Gully cricket practice match 👌🥰😘❤️🔥#cricket #c...
3       IPL 2023: Jasprit Bumrah out of IPL 2023 due t...
4       Play #AndarBahar Live game with 100% Bonus on ...
                              ...                        
1495    ಎಸ್‌ಆರ್‌ಎಚ್‌ ತಂಡದಲ್ಲಿ ಬದಲಾವಣೆ ಪರ್ವ; ಯಾರಿಗೆ ಕೊಕ...
1496    #YahanIndiaBanayegaTeam #fantasy #ipl #tataipl...
1497    ಸನ್‌ರೈಸರ್ಸ್‌ ಹೈದರಬಾದ್ ತಂಡದ ನೂತನ ಕೋಚ್ ಆಗಿ ಬ್ರಿಯ...
1498    IPL: Sunrisers Hyderabad part ways with Tom Mo...
1499    Sunrisers Hyderabad: 'हा' दिग्गज खेळाडू बनला S...
Name: text, Length: 1500, dtype: object


C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=remove_url(df['text'][i])
C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=emoji.demojize(df['text'][i])
C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=remov

  0%|          | 0/117 [00:00<?, ?it/s]

c:\Users\anik3\anaconda3\lib\site-packages\transformers\generation\utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 64 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


### 3Idiots

In [102]:
df=pd.read_csv("3idiots.csv")
print(df.text)
preprocess(df)
newdf=buildsetoften(df)
for i in tqdm(range (len(newdf)),total=len(newdf)):
    t=""
    # print(newdf[i])
    input.append(newdf[i])
    # print("summary")
    t=generate_summary(newdf[i])
    # print(t)
    output.append(t)

0       #throwback from the #looktests for #3Idiots 😍❤...
1       @KritiAdmirers @Fossil Haila meri to 150 rs ki...
2       3 idiots- GIF of the day! #alliswell #amir #3i...
3       @TausiM14 @iamsrk @oliviamunn @E_oniverse @kas...
4       @RajkumarHirani my roommate and I were discuss...
                              ...                        
1495    @ActorMadhavan My favorite actor from south .....
1496    😂😂😂\n.\n.\n.\n#memes #meme #maharashtra #nagpu...
1497    Sooooooooo cute #Aamirkhan \n#3idiots #Kareena...
1498    Legendary movies from legendary man.💫✨️🔥🔥\n#Aa...
1499    Agency life feat. 3 idiots \nShare if you rela...
Name: text, Length: 1500, dtype: object


C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=remove_url(df['text'][i])
C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=emoji.demojize(df['text'][i])
C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=remov

  0%|          | 0/110 [00:00<?, ?it/s]

c:\Users\anik3\anaconda3\lib\site-packages\transformers\generation\utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 64 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


### Galaxys23

In [103]:
df=pd.read_csv("#galaxys23.csv")
print(df.text)
preprocess(df)
newdf=buildsetoften(df)
for i in tqdm(range (len(newdf)),total=len(newdf)):
    t=""
    input.append(newdf[i])
    t=generate_summary(newdf[i])
    output.append(t)

0       @Khaya_dladla @SamsungMobileSA @gagasifm 😁😁 #G...
1                    #ShareTheEpic   #GalaxyS23 @gagasifm
2                     #ShareTheEpic  #GalaxyS23 @gagasifm
3                      #ShareTheEpic #GalaxyS23 @gagasifm
4       @gagasifm @SamsungMobileSA of course it does, ...
                              ...                        
1495    Follow, like and share @gagasifm and answer a ...
1496    Asiwaju Bola Ahmed Tinubu in a striking resemb...
1497    Wih keren banget hasil nya #ShareTheEpic\n#wit...
1498    فيديو 🎥:\nاستعراض وتجربة هاتف جالكسي اس 23 ، ا...
1499    @currys I can't wait to use the camera to see ...
Name: text, Length: 1500, dtype: object


C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=remove_url(df['text'][i])
C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=emoji.demojize(df['text'][i])
C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=remov

  0%|          | 0/98 [00:00<?, ?it/s]

c:\Users\anik3\anaconda3\lib\site-packages\transformers\generation\utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 64 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


### IndoChinaStandoff

In [104]:
df=pd.read_csv("#IndoChinaStandoff.csv")
print(df.text)
preprocess(df)
newdf=buildsetoften(df)
for i in tqdm(range (len(newdf)),total=len(newdf)):
    t=""
    input.append(newdf[i])
    t=generate_summary(newdf[i])
    output.append(t)

0      Boosting India's border security! Cabinet appr...
1      तवांग सेक्टर में भारतीय-चीनी सैनिकों की भिड़ंत...
2      Indian Army prevented Chinese attempts to chan...
3      LAC पर भारतीय, चीनी सैनिकों की झड़प, दोनों पक्...
4      #IndoChinaStandoff \n#EasternLadakh \n\n@IgpLa...
                             ...                        
223    Only if the money used for personal gain was u...
224    5. Instead, the Indian govt buckled under the ...
225    4. Indian govt is fooling the people by callin...
226    3.The current situation has mellowed down only...
227    2.Such transgressions have happened in the pas...
Name: text, Length: 228, dtype: object


C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=remove_url(df['text'][i])
C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=emoji.demojize(df['text'][i])
C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=remov

  0%|          | 0/18 [00:00<?, ?it/s]

c:\Users\anik3\anaconda3\lib\site-packages\transformers\generation\utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 64 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


### heartattack

In [117]:
df=pd.read_csv("#heartattack.csv")
print(df.text)
preprocess(df)
newdf=buildsetoften(df)
for i in tqdm(range (len(newdf)),total=len(newdf)):
    t=""
    input.append(newdf[i])
    t=generate_summary(newdf[i])
    output.append(t)

0       “Erythritol” sweetener linked to higher rates ...
1       ॐ\nCorrect side effects numbers reported?\n\n#...
2       https://t.co/fNRDT0Qcb9\n#heartattack #SuddenD...
3       I wonder how people with beautiful heart can g...
4       Sushmita Sen: पूर्व मिस यूनिवर्स को आया हार्ट ...
                              ...                        
1495    Short Documentary on peshware hospitality\n@Pa...
1496    Hridaya Suddha For Heart Problems\nhttps://t.c...
1497    ఆంధ్రప్రదేశ్ రాష్ట్రంలోని ఆదోనిలో మరో విషాద ఘట...
1498    ક્રિકેટ રમતાં રમતાં મોતનો વધુ એક બનાવ : ​​​​​​...
1499    #AskHinaParvezButt \n#heartattack \n#president...
Name: text, Length: 1500, dtype: object


C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=remove_url(df['text'][i])
C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=emoji.demojize(df['text'][i])
C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=remov

Exception
Exception


  0%|          | 0/110 [00:00<?, ?it/s]

c:\Users\anik3\anaconda3\lib\site-packages\transformers\generation\utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 64 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


### MSDhoni

In [120]:
# df=pd.read_csv("#msdhoni.csv")
# print(df.text)
# preprocess(df)
# newdf=buildsetoften(df)
# for i in tqdm(range (len(newdf)),total=len(newdf)):
#     t=""
#     input.append(newdf[i])
#     t=generate_summary(newdf[i])
#     output.append(t)

### Messi

In [107]:
df=pd.read_csv("#messi.csv")
print(df.text)
preprocess(df)
newdf=buildsetoften(df)
for i in tqdm(range (len(newdf)),total=len(newdf)):
    t=""
    input.append(newdf[i])
    t=generate_summary(newdf[i])
    output.append(t)

0       La prensa internacional se hizo eco de la amen...
1       @porquetendencia El narco termino en #Argentin...
2       El Estado se retiró de Rosario #ElPeorPresiden...
3       El expresidente expresó su apoyo a la familia ...
4       #AnibalFernandez te comunicaste con @nayibbuke...
                              ...                        
1495    Javier Mascherano no seguirá en Argentina sub ...
1496    #Acun #Messi #ZülfüLivaneli #AdımFarah #Ronald...
1497    Check out my new #messi on OpenSea! https://t....
1498    #OmuzOmuza\n#Messi\n#Ronaldo\n#Deprem\n#Yardım...
1499    Dayım gelecegi görmüs öz goatım #Mbappe #Messi...
Name: text, Length: 1500, dtype: object


C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=remove_url(df['text'][i])
C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=emoji.demojize(df['text'][i])
C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=remov

  0%|          | 0/49 [00:00<?, ?it/s]

c:\Users\anik3\anaconda3\lib\site-packages\transformers\generation\utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 64 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


### CR7

In [108]:
df=pd.read_csv("#CR7.csv")
print(df.text)
preprocess(df)
newdf=buildsetoften(df)
for i in tqdm(range (len(newdf)),total=len(newdf)):
    t=""
    input.append(newdf[i])
    t=generate_summary(newdf[i])
    output.append(t)

0       @Naija_PR Whenever i see Madrid winning, no ma...
1       لا يمكن للاعب ان يحقق ٥ كرات ذهبية في ظل محارب...
2       Dont miss sports at JETEOP TV OLWALEERO... ENG...
3       @Cristiano -&gt; @Benzema -&gt; @vinijr  the e...
4       Seni de unutmadık #CR7 \nBi tweet bile atmadın...
                              ...                        
1495    6مباريات \n8 اهداف \n2 اسيست \nبداية عظيمه للأ...
1496    Le but du TRIPLÉ de CR7 avec Al Nassr ! 🐐🇸🇦\n\...
1497    💯"Bring the Best Soccer Trainers into Your Hom...
1498    هاتررررريييييييكككك #CR7 🫶🏼🫶🏼🫶🏼💙💛💪 https://t.c...
1499    Hattrick Cristiano Ronaldo 🥰\n#Ronaldo \n#cris...
Name: text, Length: 1500, dtype: object


C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=remove_url(df['text'][i])
C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=emoji.demojize(df['text'][i])
C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=remov

  0%|          | 0/79 [00:00<?, ?it/s]

c:\Users\anik3\anaconda3\lib\site-packages\transformers\generation\utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 64 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


### WPL2023

In [116]:
df=pd.read_csv("#WPL2023.csv")
print(df.text)
preprocess(df)
newdf=buildsetoften(df)
for i in tqdm(range (len(newdf)),total=len(newdf)):
    t=""
    input.append(newdf[i])
    t=generate_summary(newdf[i])
    output.append(t)

0       WPL 2023: డబ్ల్యూపీఎల్‌తో అయినా సెమీస్‌ ఓటమి బ...
1       Beth Mooney is all set for #WPL 👏 👏 🏏 \n\n#wpl...
2       #JemimahRodrigues reckons that the #DelhiCapit...
3       शुरुआती स्टेज  पर जेमिमा का नाम सबसे आगे थे, ल...
4            4th March !! 💥\n    GG vs MI \n#WPL #WPL2023
                              ...                        
1995    #MumbaiIndians #OneFamily  #WPL2023 \n@ImHarma...
1996    Jemimah Rodriguez is such a vibe , long live g...
1997    Women's premiere league is starting from this ...
1998    Who will win #WPL2023 ? Please comment for Mumbai
1999    Jintimani Kalita - First Player from Assam to ...
Name: text, Length: 2000, dtype: object


C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=remove_url(df['text'][i])
C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=emoji.demojize(df['text'][i])
C:\Users\anik3\AppData\Local\Temp\ipykernel_6640\4124540592.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]=remov

Exception


  0%|          | 0/171 [00:00<?, ?it/s]

c:\Users\anik3\anaconda3\lib\site-packages\transformers\generation\utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 64 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


### Making a dictionary where key x contains all inputs and y contains summary

In [118]:
dict={'x':input,'y':output}
resultdf=pd.DataFrame(dict)
resultdf

,x,y
0,"Ahead of AyodhyaVerdict, appeal to everyone to...",<pad>Politicians and celebrities have been sha...
1,"WATCH Maulana Asghar Ali Salafi, President, Ma...","<pad>Maulana Asghar Ali Salafi, President of M..."
2,A civilizational wound is now healed. 490 year...,"<pad>""My essay on the AYODHYAVERDICT The Ayodh..."
3,As a step to impress Sonia Gandhiandamp Congre...,<pad>As the Ayodhya verdict continues to domin...
4,I would suggest Muslims of present generation ...,<pad>I would suggest Muslims of present genera...
...,...,...
1154,"On Camera, 19YearOld Dancing At Wedding Collap...",<pad>A 19-year-old man collapsed and died whil...
1155,I had doubt's on Covid vaccine from Day 1 so d...,<pad>A 19-year-old man who collapsed and died ...
1156,"anxiety frm pain, then hv docs,pharmacies, med...","<pad>I feel heartattack is in my near future, ..."
1157,heartattackAI has made predicting heart diseas...,<pad>Dozens of people have been taken to hospi...


### Saving the datafraeme to a csv file

In [119]:
resultdf.to_csv("NLPData.csv")